In [1]:
from data_utils import get_uci_data, common_processor_UCI, seed_all, california_housing_process, normalize, splitter
import numpy as np
from scipy.spatial import distance


import os
import yaml
from Experiments.EXP1.trainer import trainer, model_callByName, loss_callByName
from data_utils import get_uci_data, common_processor_UCI, seed_all, california_housing_process, normalize, splitter
from Experiments.EXP1.TestPerform import testPerform_muSigma, testPerform_kernel
import torch
import pandas as pd
from sklearn.ensemble import RandomForestRegressor




x, y = get_uci_data("boston")

x_norm, x_normalizer = normalize(x)

x = x_norm

tr_idx, te_idx = splitter(int(len(x)/2), int(len(x)/2))

# we resample from tr_idx to create a shifted distribution

raw_train_X, raw_train_Y, test_X, test_Y = x[tr_idx], y[tr_idx], x[te_idx], y[te_idx]

x_bar = np.mean(raw_train_X, axis = 0)

cov_mat = np.cov(raw_train_X.T)

x_seed = np.random.multivariate_normal(mean = x_bar, cov = cov_mat, size = 1)[0]


resample_pickers = np.random.multivariate_normal(mean = x_seed, cov = 0.3 * cov_mat, size = 10000)

picked_idx = []

for picker in resample_pickers:
    
    closest_index = distance.cdist(raw_train_X, [picker]).argmin()
    picked_idx.append(closest_index)
    
train_X = raw_train_X[np.array(picked_idx)]
train_Y = raw_train_Y[np.array(picked_idx)]

train_X, test_X = torch.Tensor(x), torch.Tensor(test_X)
train_Y, test_Y = torch.Tensor(y).to(torch.device("cuda")), torch.Tensor(test_Y).to(torch.device("cuda"))

# benchmarks

In [2]:



for modelname in [ "HNN", "MC_drop", "DeepEnsemble"]:

    # train base model
    print("model: "+ modelname +" on data: shifted boston")

    with open(os.getcwd()+"/Experiments/EXP1/config_bin/"+modelname+"_on_wine_config.yml", 'r') as file:
        base_configs = yaml.safe_load(file)

    base_misc_info = base_configs["misc_info"]
    base_train_config= base_configs["training_config"]

    base_misc_info["model_config"]["hidden_layers"] = [10, 5]
    base_misc_info["model_config"]["n_input"] = x.shape[1]

    base_train_config["LR"] = 5E-3
    base_train_config["bat_size"] = 64

    crits_dic = {}

    

    base_model = model_callByName[base_misc_info["model_init"]](**base_misc_info["model_config"])

    

    trainer(
        raw_train_X = train_X,
        raw_train_Y = train_Y,
        model = base_model,
        training_config = base_train_config,
        harvestor = None,          
        misc_info = base_misc_info,
        diff_trainingset = True
    )



    record = testPerform_muSigma(test_X, test_Y, model_name= modelname, model = base_model)

    print(record)

model: HNN on data: shifted boston
{'rmse_loss': 5.571051597595215, 'mean_std_norm_loss': 2.3370025157928467, 'MACE_muSigma': 0.09484067559242249, 'AGCE_muSigma': 0.11602626740932465, 'CheckScore_muSigma': 1.609297752380371}
model: MC_drop on data: shifted boston
{'rmse_loss': 7.239336013793945, 'mean_std_norm_loss': 2.349358558654785, 'MACE_muSigma': 0.16299006342887878, 'AGCE_muSigma': 0.21959999203681946, 'CheckScore_muSigma': 1.86886727809906}
model: DeepEnsemble on data: shifted boston
{'rmse_loss': 5.573818206787109, 'mean_std_norm_loss': 2.2600443363189697, 'MACE_muSigma': 0.08459816873073578, 'AGCE_muSigma': 0.14312119781970978, 'CheckScore_muSigma': 1.5405875444412231}


# VanillaPred

In [3]:
modelname =  "vanillaPred"

# train base model
print("model: "+ modelname +" on data: shifted boston")

with open(os.getcwd()+"/Experiments/EXP1/config_bin/"+modelname+"_on_wine_config.yml", 'r') as file:
    base_configs = yaml.safe_load(file)

base_misc_info = base_configs["misc_info"]
base_train_config= base_configs["training_config"]

base_misc_info["model_config"]["hidden_layers"] = [10, 5]
base_misc_info["model_config"]["n_input"] = x.shape[1]

base_train_config["LR"] = 5E-3
base_train_config["bat_size"] = 64

crits_dic = {}



pred_model = model_callByName[base_misc_info["model_init"]](**base_misc_info["model_config"])

N_train = int(len(train_X)*0.7)
N_recal = int(len(train_X)*0.3)

tr_idx, recal_idx = splitter(N_train, N_recal)



trainer(
    raw_train_X = train_X[tr_idx],
    raw_train_Y = train_Y[tr_idx],
    model = pred_model,
    training_config = base_train_config,
    harvestor = None,          
    misc_info = base_misc_info,
    diff_trainingset = True
)

model: vanillaPred on data: shifted boston


In [4]:
width = 10

record = testPerform_kernel(
        test_X, test_Y, train_X[recal_idx], train_Y[recal_idx], 
        model_name = "vanillaKernel", model= pred_model, wid = width)
print(record)               

{'MACE_Loss': 0.02700219117105007, 'AGCE_Loss': 0.11656970530748367, 'CheckScore': 0.9331902861595154}
